In [ ]:
# from Modules.LSH import semantic_query_lsh
# from Modules.LSH import LSH


# import numpy as np
# import os

In [ ]:
# file_path = "./random_data.txt"
# read_data = np.loadtxt(file_path)
# plane_norms = LSH(read_data, 8)
# query=[read_data[0]]
# folder_name = "bucket_files"
# result = semantic_query_lsh(query, plane_norms,folder_name)

In [159]:
%load_ext autoreload
%autoreload 2


from utils import *
from Modules.LSH import*
from api import *
from evaluation import *
from worst_case_implementation import VecDBWorst


# datafile_path="../DataBase/random_data_10000.txt"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data_file='../DataBase/data.bin'
Level_1_path='../DataBase/Level1'
Level_2_path='../DataBase/Level2'
Level_3_path='../DataBase/Level3'

Level_1_nbits=8
Level_2_nbits=3
Level_3_nbits=3

data_api = DataApi(data_file)
data_api.generate_data_file(10000)

In [ ]:

# Test LSH_index
# Read Data From File
read_data = data_api.get_first_k_records(10000)


# Layer(1)
level_1_in=read_data
# TODO: Save Planes to be used in query Search
level_1_planes=LSH_index(data=level_1_in, nbits=Level_1_nbits,index_path=Level_1_path)

In [ ]:
# Layer(2)
# On Each Bucket Apply LSH

# List all files in the directory
files = os.listdir(Level_1_path)

# TODO: Save Planes to be used in query Search
level_2_planes={}

# Loop over the files
for file_name in files:
    file_path = os.path.join(Level_1_path, file_name)
    
    if os.path.isfile(file_path):
        # Read Data
        read_data_2 = np.loadtxt(file_path,dtype=int,ndmin=1)

        level_2_in=data_api.get_multiple_records_by_ids(read_data_2)
        # level_2_in = array_to_dictionary(values=vectors,keys=np.hstack(read_data_2))

        # # Apply LSH on this Bucket
        # level_2=arr[level_1]
        level_2_planes[file_name[:-4]]=LSH_index(data=level_2_in.values(), nbits=Level_2_nbits,index_path=Level_2_path+'/' + file_name[:-4])



In [ ]:
# Layer(3)
# On Each Bucket Apply LSH

# List all files in the directory
folders = os.listdir(Level_2_path)

# TODO: Save Planes to be used in query Search
level_3_planes={}
# file_3=folder{}
# Loop over the folders
for folder_name in folders:
    level_3_planes[folder_name]={}
    folder_path = os.path.join(Level_2_path, folder_name)
    files = os.listdir(folder_path)
    # Loop over the files
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path):
            # Read Data
            read_data_3 = np.loadtxt(file_path,dtype=int,ndmin=1)

            level_3_in=data_api.get_multiple_records_by_ids(read_data_3)

            # # Apply LSH on this Bucket
            level_3_planes[folder_name][file_name[:-4]]=LSH_index(data=level_3_in.values(), nbits=Level_3_nbits,index_path=Level_3_path+'/'+folder_name+'/' + file_name[:-4])


In [161]:
# Query
query=data_api.get_record_by_id(5)[5]['embed']
# Layer (1)
bucket_1,index_result_1 = semantic_query_lsh(query=query,plane_norms=level_1_planes,index_path=Level_1_path)
print("bucket of level 1: ",bucket_1)
# print("Length of level 1",len(index_result_1))
# print("Indices of level 1",index_result_1)
print("=====================================")

# Layer(2)
bucket_2,index_result_2 = semantic_query_lsh(query=query,plane_norms=level_2_planes[bucket_1],index_path=Level_2_path+"/"+bucket_1)
print("bucket of level 2: ",bucket_2)
# print("Length of level 2",len(index_result_2))
# print("Indices of level 2",index_result_2)
print("=====================================")

# Layer(3)
bucket_3,index_result_3 = semantic_query_lsh(query=query,plane_norms=level_3_planes[bucket_1][bucket_2],index_path=Level_3_path+"/"+bucket_1+'/'+bucket_2)
print("bucket of level 3: ",bucket_3)
print("Length of level 3",len(index_result_3))
print("Indices of level 3",index_result_3)


print("=====================================")

# get top 10 results from the last layer
top_result,_=get_top_k_similar(query,index_result_3,10)
print("Top k results: ",top_results[0])
print("=====================================")
# get the intersection of the two lists level 2 and level3
count = sum(element in index_result_2 for element in index_result_3)
print("Intersection of the two layers: ",count)



bucket of level 1:  11000101
bucket of level 2:  000
bucket of level 3:  101
Length of level 3 164
Indices of level 3 [   5  117  119  141  177  184  219  220  228  281  297  365  394  658
  680  689  772  791  808  841  874  997 1029 1069 1187 1365 1407 1425
 1478 1587 1642 1672 1675 1707 1708 1822 1915 1989 2166 2206 2266 2349
 2455 2483 2527 2696 2744 2746 2862 2937 2966 2967 2989 3029 3143 3150
 3154 3170 3324 3417 3457 3460 3546 3557 3624 3625 3642 3682 3685 3719
 3727 3743 3948 4186 4277 4322 4366 4381 4425 4437 4581 4663 4696 4712
 4957 5023 5064 5137 5190 5219 5234 5272 5287 5341 5360 5361 5385 5538
 5736 5798 5816 5924 5991 6012 6024 6060 6182 6200 6223 6231 6253 6258
 6361 6373 6407 6498 6608 6666 6885 7164 7274 7276 7429 7477 7529 7584
 7589 7617 7794 7833 7851 7929 8108 8127 8233 8347 8358 8365 8447 8472
 8675 8746 8757 8796 8865 8876 8887 8978 9184 9251 9355 9394 9411 9445
 9492 9550 9562 9619 9722 9743 9812 9858 9927 9929]
Top k results:  [119  75 128  97 124 141 142   1 

In [170]:
np_rows = np.array([record['embed'] for record in read_data if 'embed' in record])
# temp=[5, 966, 536, 1088, 5073, 5549]
index_result_3_minus_one = [id - 1 for id in top_results[0]]
res=run_queries(index_result_3_minus_one, np_rows, top_k=len(top_results), num_runs=1,query=np.array([query]))
print(eval(res))

Ours: [118, 74, 127, 96, 123, 140, 141, 0, 121, 21]
His: [4, 1095, 6957, 4913, 5301, 2279, 6165, 5647, 8272, 1726, 2303, 7582, 5326, 7761, 9881, 6808, 7512, 1373, 3888, 7577, 367, 278, 712, 3947, 7657, 6599, 2294, 6302, 8283, 3738, 7038, 6214, 1777, 6938, 8005, 8379, 1973, 7958, 1967, 8107, 9762, 5649, 5283, 8922, 580, 3346, 8279, 3412, 2326, 8926, 8346, 9987, 5780, 296, 4988, 3479, 1658, 6032, 3444, 1188, 3471, 374, 1479, 9228, 7733, 8606, 110, 8373, 6128, 7933, 3449, 4653, 4379, 651, 1488, 4009, 4092, 9084, 3134, 2779, 7171, 4827, 5873, 7617, 2583, 9584, 1399, 9623, 1359, 9162, 3097, 1495, 8442, 2794, 9822, 109, 9341, 3975, 2992, 2068, 2021, 3582, 6508, 9277, 3042, 187, 1696, 3197, 692, 3484, 2229, 7670, 3535, 3220, 2874, 6608, 9326, 6234, 6237, 2149, 4486, 3640, 4627, 2632, 371, 428, 3436, 5361, 1549, 7635, 1508, 1543, 4500, 9223, 6177, 8498, 6139, 6310, 3538, 8526, 6545, 654, 1736, 3192, 1862, 5533, 9240, 5194, 2100, 1982, 8799, 1336, 5006, 9374, 3787, 4967, 7644, 2861, 5403, 9758,

In [ ]:
db = VecDBWorst()
# records_np = np.random.random((10000, 70))
records_np = np.array([record['embed'] for record in read_data if 'embed' in record])

# records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(records_np)]
records_dict=read_data

# _len = len(records_np)
db.insert_records(records_dict)
db_ids=db.retrive(query, top_k=1)
db_ids_minus_one = [id - 1 for id in db_ids]
res = run_queries(db_ids_minus_one, records_np, 1, 1, np.array([query]))
print(eval(res))
